# Project: Mashing up two APIs to make movie recommendations

This project will take you through the process of mashing up data from two different APIs to make movie recommendations. The TasteDive API lets you provide a movie (or bands, TV shows, etc.) as a query input, and returns a set of related items. The OMDB API lets you provide a movie title as a query input and get back data about the movie, including scores from various review sites (Rotten Tomatoes, IMDB, etc.).

You will put those two together. You will use TasteDive to get related movies for a whole list of titles. You’ll combine the resulting lists of related movies, and sort them according to their Rotten Tomatoes scores (which will require making API calls to the OMDB API.)

To avoid problems with rate limits and site accessibility, we have provided a cache file with results for all the queries you need to make to both OMDB and TasteDive. Just use requests_with_caching.get() rather than requests.get(). If you’re having trouble, you may not be formatting your queries properly, or you may not be asking for data that exists in our cache. We will try to provide as much information as we can to help guide you to form queries for which data exists in the cache.

Your first task will be to fetch data from TasteDive. The documentation for the API is at https://tastedive.com/read/api.

Define a function, called get_movies_from_tastedive. It should take one input parameter, a string that is the name of a movie or music artist. The function should return the 5 TasteDive results that are associated with that string; be sure to only get movies, not other kinds of media. It will be a python dictionary with just one key, ‘Similar’.

Try invoking your function with the input “Black Panther”.

HINT: Be sure to include only q, type, and limit as parameters in order to extract data from the cache. If any other parameters are included, then the function will not be able to recognize the data that you’re attempting to pull from the cache. Remember, you will not need an api key in order to complete the project, because all data will be found in the cache.


In [ ]:
import requests_with_caching
import json

def get_movies_from_tastedive (str): # gets a list of movies, returns response obj 
    baseurl="https://tastedive.com/api/similar"
    params_dict={}
    params_dict["q"]=str #consists of a series (at least one) of bands, movies, TV shows, podcasts, books, authors and/or games, separated by commas
    params_dict["type"]="movies"
    params_dict["limit"]=5
    tasdive_resp=requests_with_caching.get(baseurl, params=params_dict)
    print(tasdive_resp.url)
    return tasdive_resp.json()

def extract_movie_titles(tasdive_resp):
    mov_results=tasdive_resp["Similar"]["Results"]
    mov_titles=[titles['Name'] for titles in mov_results]
    return mov_titles

def get_related_titles(mov_titles): #gets a list of movie titles, returns all related movie titles
    rel_mov=[]
    all_relmov=[]
    for mov in mov_titles:
        rel_mov=extract_movie_titles(get_movies_from_tastedive(mov))
        for item in rel_mov:
            if item not in all_relmov:
                all_relmov.append(item)
    return all_relmov  

def get_movie_data(title): #gets one title, returns info
    baseurl="http://www.omdbapi.com/"
    params_dict={}
    params_dict["t"]=title
    params_dict["r"]="json"
    resp_omdb=requests_with_caching.get(baseurl, params=params_dict)
    print(resp_omdb.url)
    return resp_omdb.json()

def get_movie_rating(resp_omdb): #gets omdb response and returns rotten tomato rating
    try:
        lst=[item['Value'] for item in resp_omdb["Ratings"] if item['Source']=='Rotten Tomatoes']
        return (int(lst[0][:-1]))
    except:
        return 0
    
def get_sorted_recommendations(mov_title_lst):
    all_rl_mov=get_related_titles(mov_title_lst)
    ratings=[get_movie_rating(get_movie_data(title)) for title in all_rl_mov]
    print(all_rl_mov)
    print(ratings)
    sort_mov=[mov for rat, mov in sorted(zip(ratings,all_rl_mov), key=lambda x: (x[0], x[1]),reverse=True)]
    #print(sort_mov)
    return(sort_mov)

# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
print(get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"]))


# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
# get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"])

